In [117]:
import cudf as pd
import cupy as np

In [118]:
house = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
house_test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
house.head()

In [119]:
total = house.isnull().sum().sort_values(ascending=False)
percent = (house.isnull().sum()/house.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1)
missing_data.head(20)


In [120]:
rem_com = missing_data.index.to_arrow()[0:18]
rem_com = rem_com.to_pylist()
rem_com

In [121]:
house = house.drop(rem_com[:],axis=1)
house = house.drop(house.loc[house['Electrical'].isnull()].index)
house.isnull().sum().max()

In [122]:
house

# Label Encoding

In [123]:
all_col = house.columns.tolist()
num_col = house._get_numeric_data().columns.tolist()
categorical_col = list(set(all_col) - set(num_col))
print(len(categorical_col))

In [124]:
from cuml.preprocessing import LabelEncoder as le
for c in categorical_col:
    house[c] = le().fit_transform(house[c])

print(house[categorical_col])

**Doing same on Test Data**

In [125]:
house_test = house_test.drop(rem_com[:],axis=1,)
for c in categorical_col:
    house_test[c]=le().fit_transform(house_test[c])


**Using Forward fill for Filling Missing Values**

In [126]:
for c in all_col[:-1]:
        house_test[c]=house_test[c].fillna(method='ffill')
house_test.isnull().sum().sort_values(ascending=False)

In [127]:
house=house.astype('float64')
house_test=house_test.astype('float64')
X = house.iloc[:,:-1]
Y = house.iloc[:,-1]

In [128]:
print(X.info(), Y)

# Making Files for each Algorithm on The Test Data

In [134]:
from cuml import LinearRegression
algorithm = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']

for i in algorithm:
    lr = LinearRegression(algorithm = i)
    lr.fit(X,Y)
    predicted = lr.predict(house_test)
    predicted=predicted.astype('Int64')
    predicted=predicted.to_arrow().to_pylist()
    predicted_df=pd.DataFrame(data=predicted,columns=['SalePrice'])
    results = pd.concat([house_test["Id"].astype('Int64'), predicted_df], axis=1)
    results.to_csv('/kaggle/working/submission-'+inp+'.csv',index=False)


# Comparing Different algorithms by splitting Train Data

In [135]:
import cuml
from cuml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state = 2)

for i in algorithm:
    print("Algorithm:")
    print(i)
    lr = LinearRegression(fit_intercept = True, normalize = False, algorithm = i)
    regressor = lr.fit(X_train.astype('float32'), y_train.astype('float32'))
    predicts = lr.predict(X_test)
    print("MSE:")
    print(cuml.metrics.regression.mean_squared_error(y_test.astype('int64'),predicts.astype('int64')))
    print("R2 Score:")
    print(cuml.metrics.regression.r2_score(y_test.astype('float32'), predicts.astype('float32')))
    print("MAE:")
    print(cuml.metrics.regression.mean_absolute_error(y_test.astype('int64'), predicts.astype('int64')))
    print('\n')
    print('---------------')